# 文本生成实战

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


## 数据读取

In [3]:
input_filepath = './data/shakespeare.txt'
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### 数据处理
1. 生成词表 generator vocab
2. 建立字符到id的映射 build mapping char->id
3. 将数据转化为id data->id_data
4. 预测下一个字符的模型，abcd -> bcd\<eos>

In [4]:
# 1.generator vocab
vocab = sorted(set(text)) # 把字符串存为set，则只剩下唯一的字符
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# 2.build mapping char->id
char2idx = {char : idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [6]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
# 3.data->id_data
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [8]:
# 4.定义输入输出
def split_input_target(id_text):
    """abcde -> abcd, bcde"""
    return id_text[0:-1], id_text[1:]

# 定义dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1, drop_remainder = True)

# test
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])
    
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [9]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [10]:
# 给dataset做变换
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## 模型构建

In [11]:
vocab_size = len(vocab) # 词表大小
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
        # 每一步都要返回全部序列，以便和输出的序列做loss计算
        # 调参：stateful = True , recurrent_initializer = 'glorot_uniform'
        keras.layers.LSTM(units=rnn_units, stateful = True,
                          recurrent_initializer = 'glorot_uniform', return_sequences=True),
        keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### 采样测试

In [12]:
# 没经过训练，已经可以进行预测，看一下model的输出
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [13]:
# 随机采样 random sampling 或者贪心采样 greedy 获得字符，从而生成一段话
sample_indices = tf.random.categorical(logits=example_batch_predictions[0], num_samples=1)
print(sample_indices) # 采样后的维度 (100, 65) -> (100, 1)

# 将维度展平
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[48]
 [ 2]
 [31]
 [ 5]
 [33]
 [54]
 [ 1]
 [48]
 [ 1]
 [36]
 [26]
 [ 6]
 [13]
 [48]
 [37]
 [41]
 [59]
 [38]
 [46]
 [60]
 [40]
 [27]
 [38]
 [54]
 [50]
 [39]
 [41]
 [61]
 [53]
 [ 7]
 [ 3]
 [45]
 [57]
 [16]
 [62]
 [45]
 [17]
 [ 5]
 [59]
 [14]
 [44]
 [58]
 [58]
 [62]
 [ 5]
 [ 4]
 [30]
 [16]
 [ 5]
 [11]
 [54]
 [42]
 [62]
 [47]
 [48]
 [53]
 [57]
 [40]
 [28]
 [28]
 [53]
 [62]
 [26]
 [47]
 [52]
 [57]
 [47]
 [ 6]
 [ 9]
 [ 6]
 [ 9]
 [57]
 [23]
 [62]
 [18]
 [10]
 [ 9]
 [48]
 [27]
 [59]
 [23]
 [ 2]
 [61]
 [ 6]
 [62]
 [ 2]
 [14]
 [23]
 [ 1]
 [16]
 [39]
 [ 3]
 [50]
 [32]
 [39]
 [18]
 [49]
 [14]
 [ 1]
 [ 7]], shape=(100, 1), dtype=int64)
tf.Tensor(
[48  2 31  5 33 54  1 48  1 36 26  6 13 48 37 41 59 38 46 60 40 27 38 54
 50 39 41 61 53  7  3 45 57 16 62 45 17  5 59 14 44 58 58 62  5  4 30 16
  5 11 54 42 62 47 48 53 57 40 28 28 53 62 26 47 52 57 47  6  9  6  9 57
 23 62 18 10  9 48 27 59 23  2 61  6 62  2 14 23  1 16 39  3 50 32 39 18
 49 14  1  7], shape=(100,), dtype=int64)


In [14]:
# 打印输入、输出、预测
print('Input:', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Output:', repr(''.join(idx2char[target_example_batch[0]])))
print()
print('Predictions:', repr(''.join(idx2char[sample_indices])))

Input: "ust not simple Henry nor his oaths.\n\nMONTAGUE:\nBrother, I go; I'll win them, fear it not:\nAnd thus m"

Output: "st not simple Henry nor his oaths.\n\nMONTAGUE:\nBrother, I go; I'll win them, fear it not:\nAnd thus mo"

Predictions: "j!S'Up j XN,AjYcuZhvbOZplacwo-$gsDxgE'uBfttx'&RD';pdxijosbPPoxNinsi,3,3sKxF:3jOuK!w,x!BK Da$lTaFkB -"


### 训练

In [15]:
# 自定义损失函数，loss从logits里来
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)

example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1749406


In [16]:
# 定义文件保存模型
output_dir = 'text_generation_lstm'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')

# 定义参数
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix, save_weights_only = True)
epochs = 100
callbacks = [
    checkpoint_callback,
    keras.callbacks.EarlyStopping(patience=10, min_delta=1e-3)
]

# 训练
history = model.fit(seq_dataset, epochs = epochs, callbacks = callbacks)

Epoch 1/100
    172/Unknown - 56s 324ms/step - loss: 2.5390WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
172/172 [==============================] - 57s 331ms/step - loss: 2.5390
Epoch 2/100
172/172 [==============================] - 55s 320ms/step - loss: 1.8547
Epoch 3/100
172/172 [==============================] - 55s 319ms/step - loss: 1.6151
Epoch 4/100
172/172 [==============================] - 55s 321ms/step - loss: 1.4879
Epoch 5/100
172/172 [==============================] - 56s 324ms/step - loss: 1.4101
Epoch 6/100
172/172 [==============================] - 60s 351ms/step - loss: 1.3551
Epoch 7/100
172/172 [==============================] - 61s 352ms/step - loss: 1.3107
Epoch 8/100
172/172 [==============================] - 61s 353ms/step - loss: 1.2714
Epoch 9/100
172/172 [==============================] - 65s 378ms/step - loss: 1.2349
Epoch 10/100
172/172 [==============================] - 66s 386ms/ste

In [17]:
tf.train.latest_checkpoint(output_dir)

'text_generation_lstm\\ckpt_100'

## 载入模型权重，进行文本生成
文本生成的流程：
- start ch sequence A
- A -> model -> b
- A.append(b) -> B
- B(Ab) -> model -> c
- B.append(c) -> C
- C(Abc) -> model -> ...一直循环结束

In [18]:
model2 = build_model(
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size = 1)
# 加载权重
model2.load_weights(tf.train.latest_checkpoint(output_dir))
# 定义输入大小
model2.build(tf.TensorShape([1, None]))

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 默认生成1000个字符的文本
def generate_text(model, start_string, num_generate = 1000):
    # 字符变id
    input_eval = [char2idx[ch] for ch in start_string]
    # 扩展一个维度，一维变二维
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    # temperature > 1, more random
    # temperature < 1, more greedy
    temperature = 0.5
    
    # 生成流程：
    # 1. model inference -> predictions
    # 2. sample -> ch -> text_generated.
    # 3. update input_eval
    for _ in range(num_generate):
        # 模型预测 predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions softmax:e^xi
        # eg: 4,2 e^4/(e^4+e^2) = 0.88, e^2/(e^4+e^2) = 0.12
        # 除以2后eg: 2,1 e^2/(e^2+e) = 0.73, e/(e^2+e) = 0.27
        # 乘2后，相反
        predictions = predictions / temperature
        # 将第0维消掉
        predictions = tf.squeeze(predictions, 0)
        # 随机采样，predictions_ids: [input_eval_len, 1]
        # 取出预测值中的最后一个字符 a b c -> b c d ,所以取出d
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # 将input_eval替换成predictions，并扩维
        input_eval = tf.expand_dims([predicted_id], 0)
        
    return start_string + ''.join(text_generated)

In [20]:
new_text = generate_text(model2, 'All:')
print(new_text)

All:
All is a plague did over the tempter,
Even to the guilt or this untalued dam
Which he the sweet silence to a forget other hungs.

ROMEO:
Why, then the mere it didst thou know thist woman:
And I be goned at the great Aufidius
A shining stark and safe, and that the world slain.

KING RICHARD III:
A horse! a horse! my kingdom for a horse!

RICHMOND:
God and your father gave thee too remiss;
Whilst Bolingbroke, the priest should be content
To put as where it seems to be achieved.
The breathliot that o'er this most peril of your daughter
itself into the triumph day,
Which made the false seems than starve, 'tis but a man's;
And here I stand, both that his son a strength calls and dooms.

CAMILLO:
I pray not, and all this is long of brother,
A femiod dignities and mild behavior,
Am bold to show myself a forward guest
Within your house, with no man hearing how fares our cousin;
Had he done so to well and think they grow:
How far is it that she shall have me at the Tower?

BUCKINGHAM:
It i